##### Import

In [1]:
import warnings
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm
import shap
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.pipeline import Pipeline
import os
import gc
import sys

# Filter out warning messages
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

# Set seaborn style
sns.set_style('whitegrid')

# Add the parent directory to sys.path
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Index and deciles for data slicing
idx = pd.IndexSlice

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
# from pathlib import Path
# import pandas as pd
# from utils import rank_stocks_and_quantile
# # UNSEEN_KEY = '/data/YEAR_20220803_20230803'
# top = 250  # parameters -> papermill
# DATA_STORE = Path(f'data/{top}_dataset.h5')
# with pd.HDFStore(DATA_STORE) as store:
#     # unseen = store[UNSEEN_KEY]
#     print(store.keys())

In [3]:
import gc
import pandas as pd
from utils import rank_stocks_and_quantile

top = 250  # parameters -> papermill

DATA_STORE = Path(f'data/{top}_dataset.h5')
dataset_keys = [
    '/data/YEAR_20200930_20220802',
    # '/data/YEAR_20181024_20200929',
    # '/data/YEAR_20161116_20181023',
    # '/data/YEAR_20141210_20161115'
]
target_string = 'TARGET_ret_fwd'  # no longer a parameter

# Initialize empty dataset
dataset = pd.DataFrame()

with pd.HDFStore(DATA_STORE) as store:
    for key in dataset_keys:
        df = store[key]
        dataset = pd.concat([dataset, df], ignore_index=False)
        del df
        gc.collect()  # Explicitly call garbage collector

# Rank stocks and quantile
dataset = rank_stocks_and_quantile(dataset, target_substring=target_string)

# Adjust timezone
dataset.index.set_levels(dataset.index.levels[0].tz_localize(None), level=0, inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 117250 entries, (Timestamp('2020-09-30 00:00:00'), 'AA') to (Timestamp('2022-08-02 00:00:00'), 'ZTS')
Columns: 622 entries, FEATURE_open to TARGET_ret_fwd_252d_rank_quantiled
dtypes: float32(360), float64(43), int32(198), int64(12), int8(9)
memory usage: 300.3+ MB


In [4]:
features = [col for col in dataset.columns if col.startswith('FEATURE_')]
target = [col for col in dataset.columns if col.startswith('TARGET_')]

In [5]:
dataset[features]

FEATURE_open  FEATURE_high  FEATURE_low  FEATURE_close  \
date       ticker                                                           
2020-09-30 AA         11.569100     11.985600    11.449900      11.519500   
           AAL        12.320000     12.915000    12.220000      12.290000   
           AAPL      112.248001    115.677002   112.078003     114.232002   
           ABBV       78.832001     80.137398    78.659302      79.365501   
           ABT       102.320000    105.679001   102.082001     105.044998   
...                         ...           ...          ...            ...   
2022-08-02 WYNN       62.580002     65.144997    62.299999      64.410004   
           X          23.387199     23.456699    22.690500      22.879601   
           XOM        93.029900     93.530998    92.175201      92.420799   
           YUM       121.287003    121.880997   120.633003     120.980003   
           ZTS       179.438995    181.175003   177.233002     178.451004   

                   FEATURE_volume  FEATURE_market_cap  FEATURE_sector  \
date       ticker                                                       
2020-09-30 AA        5.566026e+06        5.703249e+09             4.0   
           AAL       6.542813e+07        1.022514e+10             0.0   
           AAPL      1.446370e+08        2.779610e+12             3.0   
           ABBV      9.060012e+06        2.686048e+11             2.0   
           ABT       9.827987e+06        1.827505e+11             2.0   
...                           ...                 ...             ...   
2022-08-02 WYNN      1.913168e+06        1.144238e+10             0.0   
           X         9.316298e+06        5.065737e+09             4.0   
           XOM       1.702091e+07        4.476770e+11             5.0   
           YUM       2.035008e+06        3.760435e+10             0.0   
           ZTS       1.182675e+06        8.818486e+10             2.0   

                   FEATURE_ret_frac_order  FEATURE_ret_01d  FEATURE_ret_02d  \
date       ticker                                                             
2020-09-30 AA                    1.163237        -0.003452        -0.049836   
           AAL                   0.809530         0.003265        -0.036834   
           AAPL                 16.058228         0.015071         0.007390   
           ABBV                  5.939329         0.008046         0.003534   
           ABT                  13.630976         0.034437         0.042072   
...                                   ...              ...              ...   
2022-08-02 WYNN                  9.541147         0.013692         0.014650   
           X                     5.294522        -0.034033        -0.027910   
           XOM                  10.034766        -0.004338        -0.029506   
           YUM                   9.541129        -0.004968        -0.003345   
           ZTS                  13.821751        -0.005062        -0.020377   

                   FEATURE_ret_03d  FEATURE_ret_04d  FEATURE_ret_05d  \
date       ticker                                                      
2020-09-30 AA             0.006940        -0.006006        -0.000859   
           AAL            0.000000         0.044180         0.030176   
           AAPL           0.031328         0.070149         0.081149   
           ABBV           0.015765         0.019566         0.005167   
           ABT            0.051893         0.078331         0.059250   
...                            ...              ...              ...   
2022-08-02 WYNN           0.011940         0.017696         0.051592   
           X              0.059447         0.082392         0.126963   
           XOM            0.015437         0.027302         0.049537   
           YUM           -0.004722         0.013445         0.036409   
           ZTS           -0.018875         0.007663         0.013892   

                   FEATURE_ret_10d  FEATURE_ret_21d  FEATURE_ret_42d  \
date       ticker             